In [1]:
import keras.backend as K
from keras.layers import Dense, Input
from keras.models import Model
from keras.optimizers import SGD, RMSprop
from keras.engine.topology import Layer, InputSpec
from keras.initializers import VarianceScaling
import numpy as np
from sklearn import metrics

Using TensorFlow backend.


In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/eki/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
from nltk.stem.snowball import GermanStemmer
from nltk.corpus import stopwords

In [4]:
facts = []
with open('data/facts.txt') as f:
    for line in f:
        facts.append(line.rstrip())

In [5]:
stemmer = GermanStemmer()
def snowball_stemmer(doc):
    return [stemmer.stem(word) for word in doc]

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(tokenizer=snowball_stemmer,
                        stop_words=stopwords.words('german'),
                        max_df=0.5,
                        lowercase=True,
                        ngram_range=(1,3))
features = tfidf.fit_transform(facts)

In [7]:
features.shape

(53, 7018)

In [8]:
def autoencoder(dims, act='relu', init='glorot_uniform'):
    """
    Fully connected auto-encoder model, symmetric.
    Arguments:
        dims: list of number of units in each layer of encoder. dims[0] is input dim, dims[-1] is units in hidden layer.
            The decoder is symmetric with encoder. So number of layers of the auto-encoder is 2*len(dims)-1
        act: activation, not applied to Input, Hidden and Output layers
    return:
        (ae_model, encoder_model), Model of autoencoder and model of encoder
    """
    n_stacks = len(dims) - 1
    # input
    inp = Input(shape=(dims[0],), name='input')
    x = inp
    # internal layers in encoder
    for i in range(n_stacks-1):
        x = Dense(dims[i + 1], activation=act, kernel_initializer=init, name='encoder_%d' % i)(x)

    # hidden layer
    encoded = Dense(dims[-1], kernel_initializer=init, name='encoder_%d' % (n_stacks - 1))(x)  # hidden layer, features are extracted from here

    x = encoded
    # internal layers in decoder
    for i in range(n_stacks-1, 0, -1):
        x = Dense(dims[i], activation=act, kernel_initializer=init, name='decoder_%d' % i)(x)

    # output
    x = Dense(dims[0], kernel_initializer=init, name='decoder_0')(x)
    decoded = x
    return Model(inputs=inp, outputs=decoded, name='AE'), Model(inputs=inp, outputs=encoded, name='encoder')

In [9]:
x = features.todense()

In [10]:
dims = [x.shape[-1], 500, 500, 2000, 10]
init = VarianceScaling(scale=1. / 3., mode='fan_in',
                           distribution='uniform')
pretrain_optimizer = SGD(lr=1., momentum=0.9)
pretrain_epochs = 300
batch_size = 16

In [11]:
autoencoder, encoder = autoencoder(dims, init=init)

In [12]:
autoencoder.compile(optimizer=pretrain_optimizer, loss='mse')
autoencoder.fit(x, x, batch_size=batch_size, epochs=pretrain_epochs)

Epoch 1/300
53/53 [==============================] - 1s 19ms/step - loss: 1.4247e-04
Epoch 2/300
53/53 [==============================] - 0s 7ms/step - loss: 1.4234e-04
Epoch 3/300
53/53 [==============================] - 0s 7ms/step - loss: 1.4210e-04
Epoch 4/300
53/53 [==============================] - 0s 7ms/step - loss: 1.4181e-04
Epoch 5/300
53/53 [==============================] - 0s 8ms/step - loss: 1.4148e-04
Epoch 6/300
53/53 [==============================] - 0s 7ms/step - loss: 1.4114e-04
Epoch 7/300
53/53 [==============================] - 0s 8ms/step - loss: 1.4078e-04
Epoch 8/300
53/53 [==============================] - 0s 8ms/step - loss: 1.4042e-04
Epoch 9/300
53/53 [==============================] - 1s 12ms/step - loss: 1.4008e-04
Epoch 10/300
53/53 [==============================] - 0s 8ms/step - loss: 1.3973e-04
Epoch 11/300
53/53 [==============================] - 0s 7ms/step - loss: 1.3939e-04
Epoch 12/300
53/53 [==============================] - 0s 7ms/step - loss

53/53 [==============================] - 0s 7ms/step - loss: 1.2766e-04
Epoch 98/300
53/53 [==============================] - 0s 8ms/step - loss: 1.2762e-04
Epoch 99/300
53/53 [==============================] - 0s 8ms/step - loss: 1.2758e-04
Epoch 100/300
53/53 [==============================] - 0s 8ms/step - loss: 1.2754e-04
Epoch 101/300
53/53 [==============================] - 0s 7ms/step - loss: 1.2750e-04
Epoch 102/300
53/53 [==============================] - 0s 6ms/step - loss: 1.2746e-04
Epoch 103/300
53/53 [==============================] - 0s 6ms/step - loss: 1.2743e-04
Epoch 104/300
53/53 [==============================] - 0s 6ms/step - loss: 1.2739e-04
Epoch 105/300
53/53 [==============================] - 0s 6ms/step - loss: 1.2736e-04
Epoch 106/300
53/53 [==============================] - 0s 7ms/step - loss: 1.2732e-04
Epoch 107/300
53/53 [==============================] - 0s 7ms/step - loss: 1.2729e-04
Epoch 108/300
53/53 [==============================] - 0s 7ms/step - l

53/53 [==============================] - 0s 8ms/step - loss: 1.2608e-04
Epoch 193/300
53/53 [==============================] - 0s 8ms/step - loss: 1.2607e-04
Epoch 194/300
53/53 [==============================] - 0s 8ms/step - loss: 1.2607e-04
Epoch 195/300
53/53 [==============================] - 0s 7ms/step - loss: 1.2607e-04
Epoch 196/300
53/53 [==============================] - 0s 7ms/step - loss: 1.2606e-04
Epoch 197/300
53/53 [==============================] - 0s 6ms/step - loss: 1.2606e-04
Epoch 198/300
53/53 [==============================] - 0s 7ms/step - loss: 1.2605e-04
Epoch 199/300
53/53 [==============================] - 0s 7ms/step - loss: 1.2605e-04
Epoch 200/300
53/53 [==============================] - 0s 7ms/step - loss: 1.2605e-04
Epoch 201/300
53/53 [==============================] - 0s 7ms/step - loss: 1.2604e-04
Epoch 202/300
53/53 [==============================] - 0s 7ms/step - loss: 1.2604e-04
Epoch 203/300
53/53 [==============================] - 0s 8ms/step -

53/53 [==============================] - 0s 7ms/step - loss: 1.2591e-04
Epoch 288/300
53/53 [==============================] - 0s 6ms/step - loss: 1.2591e-04
Epoch 289/300
53/53 [==============================] - 1s 10ms/step - loss: 1.2591e-04
Epoch 290/300
53/53 [==============================] - 1s 11ms/step - loss: 1.2591e-04
Epoch 291/300
53/53 [==============================] - 1s 14ms/step - loss: 1.2591e-04
Epoch 292/300
53/53 [==============================] - 1s 12ms/step - loss: 1.2591e-04
Epoch 293/300
53/53 [==============================] - 1s 10ms/step - loss: 1.2591e-04
Epoch 294/300
53/53 [==============================] - 0s 8ms/step - loss: 1.2591e-04
Epoch 295/300
53/53 [==============================] - 0s 7ms/step - loss: 1.2591e-04
Epoch 296/300
53/53 [==============================] - 0s 7ms/step - loss: 1.2591e-04
Epoch 297/300
53/53 [==============================] - 0s 6ms/step - loss: 1.2591e-04
Epoch 298/300
53/53 [==============================] - 0s 8ms/s

In [13]:
#autoencoder.save_weights('ae_weights.h5')

In [14]:
class ClusteringLayer(Layer):
    """
    Clustering layer converts input sample (feature) to soft label, i.e. a vector that represents the probability of the
    sample belonging to each cluster. The probability is calculated with student's t-distribution.

    # Example
    ```
        model.add(ClusteringLayer(n_clusters=10))
    ```
    # Arguments
        n_clusters: number of clusters.
        weights: list of Numpy array with shape `(n_clusters, n_features)` witch represents the initial cluster centers.
        alpha: degrees of freedom parameter in Student's t-distribution. Default to 1.0.
    # Input shape
        2D tensor with shape: `(n_samples, n_features)`.
    # Output shape
        2D tensor with shape: `(n_samples, n_clusters)`.
    """

    def __init__(self, n_clusters, weights=None, alpha=1.0, **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        super(ClusteringLayer, self).__init__(**kwargs)
        self.n_clusters = n_clusters
        self.alpha = alpha
        self.initial_weights = weights
        self.input_spec = InputSpec(ndim=2)

    def build(self, input_shape):
        assert len(input_shape) == 2
        input_dim = input_shape[1]
        self.input_spec = InputSpec(dtype=K.floatx(), shape=(None, input_dim))
        self.clusters = self.add_weight((self.n_clusters, input_dim), initializer='glorot_uniform', name='clusters')
        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights
        self.built = True

    def call(self, inputs, **kwargs):
        """ student t-distribution, as same as used in t-SNE algorithm.
         Measure the similarity between embedded point z_i and centroid µ_j.
                 q_ij = 1/(1+dist(x_i, µ_j)^2), then normalize it.
                 q_ij can be interpreted as the probability of assigning sample i to cluster j.
                 (i.e., a soft assignment)
        Arguments:
            inputs: the variable containing data, shape=(n_samples, n_features)
        Return:
            q: student's t-distribution, or soft labels for each sample. shape=(n_samples, n_clusters)
        """
        q = 1.0 / (1.0 + (K.sum(K.square(K.expand_dims(inputs, axis=1) - self.clusters), axis=2) / self.alpha))
        q **= (self.alpha + 1.0) / 2.0
        q = K.transpose(K.transpose(q) / K.sum(q, axis=1)) # Make sure each sample's 10 values add up to 1.
        return q

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) == 2
        return input_shape[0], self.n_clusters

    def get_config(self):
        config = {'n_clusters': self.n_clusters}
        base_config = super(ClusteringLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [15]:
clustering_layer = ClusteringLayer(n_clusters=2, name='clustering')(encoder.output)
model = Model(inputs=encoder.input, outputs=clustering_layer)

In [16]:
model.compile(optimizer=SGD(lr=0.001, momentum=0.9), loss='kld')

In [17]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2, n_init=20)
y_pred = kmeans.fit_predict(encoder.predict(x))

In [19]:
y_pred_last = np.copy(y_pred)

In [20]:
model.get_layer(name='clustering').set_weights([kmeans.cluster_centers_])

In [21]:
# computing an auxiliary target distribution
def target_distribution(q):
    weight = q ** 2 / q.sum(0)
    return (weight.T / weight.sum(1)).T

In [22]:
loss = 0
index = 0
maxiter = 8000
update_interval = 140
index_array = np.arange(x.shape[0])

In [23]:
tol = 0.001 # tolerance threshold to stop training

In [24]:
y = []
with open('data/outcome.txt') as f:
    for line in f:
        y.append(int(line.rstrip()))
y = np.array(y)

In [25]:
for ite in range(int(maxiter)):
    if ite % update_interval == 0:
        q = model.predict(x, verbose=0)
        p = target_distribution(q)  # update the auxiliary target distribution p

        # evaluate the clustering performance
        y_pred = q.argmax(1)
        if y is not None:
            acc = np.round(metrics.accuracy_score(y, y_pred), 5)
            nmi = np.round(metrics.normalized_mutual_info_score(y, y_pred), 5)
            ari = np.round(metrics.adjusted_rand_score(y, y_pred), 5)
            loss = np.round(loss, 5)
            print('Iter %d: acc = %.5f, nmi = %.5f, ari = %.5f' % (ite, acc, nmi, ari), ' ; loss=', loss)

        # check stop criterion - model convergence
        delta_label = np.sum(y_pred != y_pred_last).astype(np.float32) / y_pred.shape[0]
        y_pred_last = np.copy(y_pred)
        if ite > 0 and delta_label < tol:
            print('delta_label ', delta_label, '< tol ', tol)
            print('Reached tolerance threshold. Stopping training.')
            break
    idx = index_array[index * batch_size: min((index+1) * batch_size, x.shape[0])]
    model.train_on_batch(x=x[idx], y=p[idx])
    index = index + 1 if (index + 1) * batch_size <= x.shape[0] else 0

Iter 0: acc = 0.47170, nmi = 0.00249, ari = -0.01616  ; loss= 0
Iter 140: acc = 0.47170, nmi = 0.00249, ari = -0.01616  ; loss= 0
delta_label  0.0 < tol  0.001
Reached tolerance threshold. Stopping training.


In [26]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 7018)              0         
_________________________________________________________________
encoder_0 (Dense)            (None, 500)               3509500   
_________________________________________________________________
encoder_1 (Dense)            (None, 500)               250500    
_________________________________________________________________
encoder_2 (Dense)            (None, 2000)              1002000   
_________________________________________________________________
encoder_3 (Dense)            (None, 10)                20010     
_________________________________________________________________
decoder_3 (Dense)            (None, 2000)              22000     
_________________________________________________________________
decoder_2 (Dense)            (None, 500)               1000500   
__________